In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, SPY)   1551 non-null   float64
 1   (High, SPY)    1551 non-null   float64
 2   (Low, SPY)     1551 non-null   float64
 3   (Open, SPY)    1551 non-null   float64
 4   (Volume, SPY)  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [4]:
len(tsla)

1551

In [5]:
xly.head()

Price,Close,High,Low,Open,Volume
Ticker,XLY,XLY,XLY,XLY,XLY
Date,,,,,
2019-01-02,94.046097,94.574020,91.246207,91.745849,6840800
2019-01-03,92.009819,93.555885,91.849558,93.329633,6346000
2019-01-04,95.054817,95.752431,93.065670,93.254212,7269100
2019-01-07,97.204224,97.826417,95.460187,95.507318,6263100
2019-01-08,98.278908,98.806831,96.883674,98.212917,9391000


In [6]:
def calculate_vortex(df, value, n=14):
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [7]:
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

In [8]:
spy.head(20)

Price,Close,High,Low,Open,Volume,VI+,VI-
Ticker,SPY,SPY,SPY,SPY,SPY,,
Date,,,,,,,
2019-01-02,226.954742,227.889136,223.117435,223.144649,126925200,NaN,NaN
2019-01-03,221.538986,225.494230,221.049109,225.185783,144140700,NaN,NaN
2019-01-04,228.959610,229.612771,224.224203,224.605211,142628800,NaN,NaN
2019-01-07,230.764816,232.189059,228.324538,229.231704,103139100,NaN,NaN
2019-01-08,232.932938,233.422815,230.420098,232.978313,102512600,NaN,NaN
2019-01-09,234.021591,234.874329,232.406837,233.649650,95006600,NaN,NaN
2019-01-10,234.847107,235.101112,231.780882,232.470337,96823900,NaN,NaN
2019-01-11,234.937836,234.965050,233.168851,233.758503,73858100,NaN,NaN


In [9]:
import requests
import pandas as pd

url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=PNM5EHRALIOT1CKJ'

response = requests.get(url)

if response.status_code == 200:
    sentiment_data = response.json()
    sentiment_df = pd.DataFrame(sentiment_data['feed']) 
    print(sentiment_df.head())
else:
    print("API call failed:", response.status_code)


                                               title  \
0  Looking Back on Berkshire Hathaway's Outperfor...   
1                       How to Tune Out Market Noise   
2  EXCLUSIVE: Which Magnificent 7 Stock Will Perf...   
3  New Inflation Data Dismays Bulls, CoreWeave IP...   
4  Assessing Apple's Performance Against Competit...   

                                                 url   time_published  \
0  https://www.fool.com/investing/2025/03/28/look...  20250328T211200   
1  https://www.fool.com/investing/2025/03/28/how-...  20250328T194400   
2  https://www.benzinga.com/tech/25/03/44544129/e...  20250328T190544   
3  https://www.benzinga.com/markets/equities/25/0...  20250328T161558   
4  https://www.benzinga.com/insights/news/25/03/4...  20250328T150055   

               authors                                            summary  \
0  [Motley Fool Staff]  Over the past five years, Warren Buffett's ret...   
1  [Motley Fool Staff]  In this podcast, Motley Fool analyst Asit Shar

In [10]:
sentiment_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,Looking Back on Berkshire Hathaway's Outperfor...,https://www.fool.com/investing/2025/03/28/look...,20250328T211200,[Motley Fool Staff],"Over the past five years, Warren Buffett's ret...",https://g.foolcdn.com/editorial/images/812798/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.123972,Neutral,"[{'ticker': 'KTB', 'relevance_score': '0.00890..."
1,How to Tune Out Market Noise,https://www.fool.com/investing/2025/03/28/how-...,20250328T194400,[Motley Fool Staff],"In this podcast, Motley Fool analyst Asit Shar...",https://g.foolcdn.com/editorial/images/812797/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.071763,Neutral,"[{'ticker': 'DKNG', 'relevance_score': '0.0076..."
2,EXCLUSIVE: Which Magnificent 7 Stock Will Perf...,https://www.benzinga.com/tech/25/03/44544129/e...,20250328T190544,[Chris Katje],Benzinga readers pick their favorite Magnifice...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.266216,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1984..."
3,"New Inflation Data Dismays Bulls, CoreWeave IP...",https://www.benzinga.com/markets/equities/25/0...,20250328T161558,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.240809,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1725..."
4,Assessing Apple's Performance Against Competit...,https://www.benzinga.com/insights/news/25/03/4...,20250328T150055,[Benzinga Insights],Amidst the fast-paced and highly competitive b...,https://www.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.265078,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.4680..."
5,Apple's Options Frenzy: What You Need to Know ...,https://www.benzinga.com/insights/options/25/0...,20250328T134533,[Benzinga Insights],Investors with a lot of money to spend have ta...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.185008,Somewhat-Bullish,"[{'ticker': 'EVR', 'relevance_score': '0.06568..."
6,Trading The SPY As PCE Inflation Report Takes ...,https://www.benzinga.com/markets/equities/25/0...,20250328T125805,[RIPS],Good Morning Traders! In today's Market Clubho...,https://www.benzinga.com/next-assets/images/sc...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.084859,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0813..."
7,EU To Issue Minimal Fines To Apple And Meta To...,https://www.benzinga.com/markets/25/03/4453457...,20250328T122729,[Namrata Sen],The European Union ( EU ) is reportedly set to...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.103777,Neutral,"[{'ticker': 'META', 'relevance_score': '0.2658..."
8,3 Northern Mutual Funds for Solid Returns,https://www.zacks.com/stock/news/2436613/3-nor...,20250328T111100,[Zacks Equity Research],"Invest in Northern mutual funds like NOIEX, NM...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.272886,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1496..."
9,2 Breakout Growth Stocks You Can Buy and Hold ...,https://www.fool.com/investing/2025/03/28/2-br...,20250328T110700,[Anders Bylund],Looking for undervalued stocks with explosive ...,https://g.foolcdn.com/editorial/images/812691/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary',

## volatility

In [11]:

# Flatten MultiIndex columns 
tsla.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in tsla.columns
]

# Calculate True Range
tsla["prev_close"] = tsla["Close_TSLA"].shift(1)
tsla["tr1"] = tsla["High_TSLA"] - tsla["Low_TSLA"]
tsla["tr2"] = abs(tsla["High_TSLA"] - tsla["prev_close"])
tsla["tr3"] = abs(tsla["Low_TSLA"] - tsla["prev_close"])

tsla["true_range"] = tsla[["tr1", "tr2", "tr3"]].max(axis=1)

# 10-day ATR
tsla["ATR_10"] = tsla["true_range"].rolling(window=10).mean()

# ---- STEP 4: Calculate ATR as a percentage of closing price ----
tsla["atr_pct"] = tsla["ATR_10"] / tsla["Close_TSLA"]

# allocating the capital

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

tsla["position_size"] = tsla.apply(position_size, axis=1)

# ---- STEP 6: Optional - Capital allocation per trade ----
#capital = 100000 # Example: $100K total portfolio
#tsla["allocation_dollars"] = tsla["position_size"] * capital

# ---- Preview ----
print(tsla[["Close_TSLA", "ATR_10", "atr_pct", "position_size"]].tail(10))


            Close_TSLA     ATR_10   atr_pct  position_size
Date                                                      
2025-02-19  360.559998  16.703000  0.046325          0.005
2025-02-20  354.399994  16.464999  0.046459          0.005
2025-02-21  337.799988  17.021997  0.050391          0.005
2025-02-24  330.529999  16.770996  0.050740          0.005
2025-02-25  302.799988  18.879996  0.062351          0.005
2025-02-26  290.799988  18.412994  0.063318          0.005
2025-02-27  281.950012  18.257996  0.064756          0.005
2025-02-28  292.980011  18.067996  0.061670          0.005
2025-03-03  284.649994  19.281998  0.067739          0.005
2025-03-04  272.040009  20.654996  0.075926          0.005


In [12]:
import plotly.express as px
fig = px.line(tsla, x=tsla.index, y="atr_pct", title="ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [13]:
import plotly.express as px

# Filter only 2025 data
tsla_2025 = tsla[tsla.index.year == 2025]

# Plot
fig = px.line(tsla_2025, x=tsla_2025.index, y="atr_pct", title="ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [14]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI+_"],
    mode='lines',
    name='VI+_',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI-_"],
    mode='lines',
    name='VI-_',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for TSLA",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()


In [15]:
tsla_2025 = tsla.loc["2025"]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI+_"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI-_"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for TSLA - 2025",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [16]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for SPY",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [17]:
spy_2025 = spy.loc["2025"]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for SPY - 2025",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for XLY",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [19]:
xly_2025 = xly.loc["2025"]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for XLY - 2025",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [20]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("SPY - 2025", "XLY - 2025", "TSLA - 2025")
)

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI+"],
    name="VI+ (SPY)",
    line=dict(color='blue'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI-"],
    name="VI- (SPY)",
    line=dict(color='orange'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI+"],
    name="VI+ (XLY)",
    line=dict(color='blue'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI-"],
    name="VI- (XLY)",
    line=dict(color='orange'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI+_"],
    name="VI+ (TSLA)",
    line=dict(color='blue'),
    showlegend=False
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI-_"],
    name="VI- (TSLA)",
    line=dict(color='orange'),
    showlegend=False
), row=3, col=1)

fig.update_layout(
    height=500, width=1200,
    title_text="Vortex Indicator (VI+ and VI−) - 2025 Comparison",
    template="plotly_white"
)

fig.show()

In [21]:
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("SPY Year To Year", "XLY Year To Year", "TSLA Year To Year")
)

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI+"],
    name="VI+ (SPY)",
    line=dict(color='blue'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI-"],
    name="VI- (SPY)",
    line=dict(color='orange'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI+"],
    name="VI+ (XLY)",
    line=dict(color='blue'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI-"],
    name="VI- (XLY)",
    line=dict(color='orange'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI+_"],
    name="VI+ (TSLA)",
    line=dict(color='blue'),
    showlegend=False
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI-_"],
    name="VI- (TSLA)",
    line=dict(color='orange'),
    showlegend=False
), row=3, col=1)

fig.update_layout(
    height=900, width=1200,
    title_text="Vortex Indicator (VI+ and VI−) - Full Period Comparison",
    template="plotly_white"
)

fig.show()
